In [1]:
from colorama import Style,Fore
from dotenv import load_dotenv
import json
from tqdm import tqdm
import psycopg
import os
from datetime import datetime

load_dotenv()

True

In [ ]:
def openJson(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def saveJson(path,data):
    with open(path, "w", encoding="utf-8") as f:
       json.dump(data, f, ensure_ascii=False, indent=2)
       print(Style.BRIGHT+Fore.GREEN+'\n json saved'+Style.RESET_ALL)

# Création de base de données

In [ ]:
# Connexion à PostgreSQL (à la base par défaut, souvent 'postgres')
conn = psycopg.connect(
    dbname="postgres",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost", 
    port="5432"           
)

conn.autocommit = True
cur = conn.cursor()
nom_base = "mydatabase"
cur.execute(f"CREATE DATABASE {nom_base}")
cur.close()
conn.close()

print(f"Base de données '{nom_base}' créée avec succès.")

# Création des tables

In [ ]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)

cur = conn.cursor()

cur.execute("""
    CREATE TABLE chaines (
        id_chaine TEXT PRIMARY KEY,
        nom TEXT NOT NULL,
        bio TEXT,
        localisation CHAR(2),
        categorie_chaine TEXT,
        date_creation DATE NOT NULL,
        pertinente BOOLEAN
    )
""")

cur.execute("""
    CREATE TABLE videos (
        id_video TEXT PRIMARY KEY,
        titre TEXT NOT NULL,
        description TEXT,
        date_publication DATE NOT NULL,
        categorie_video TEXT,
        duree INTEGER,
        miniature TEXT,
        langue CHAR(2),
        transcription TEXT,
        tags TEXT[],
        requetes TEXT[],
        id_chaine TEXT REFERENCES chaines(id_chaine) ON DELETE CASCADE
    )
""")

cur.execute("""
    CREATE TABLE chaines_metriques (
        id_chaine TEXT REFERENCES chaines(id_chaine) ON DELETE CASCADE,
        date_releve_chaine DATE NOT NULL,
        nombre_vues_total INTEGER,
        nombre_abonnes_total INTEGER,
        nombre_videos_total INTEGER,
        PRIMARY KEY (id_chaine, date_releve_chaine)
    )
""")

cur.execute("""
    CREATE TABLE videos_metriques (
        id_video TEXT REFERENCES videos(id_video) ON DELETE CASCADE,
        date_releve_video DATE NOT NULL,
        nombre_vues  INTEGER,
        nombre_likes INTEGER,
        PRIMARY KEY (id_video, date_releve_video)
    )
""")

cur.execute("""
    CREATE TABLE mentions (
        id_chaine TEXT REFERENCES chaines(id_chaine) ON DELETE CASCADE,
        id_video TEXT REFERENCES videos(id_video) ON DELETE CASCADE,
        mention_titre BOOLEAN ,
        mention_tags BOOLEAN ,
        mention_description BOOLEAN ,
        PRIMARY KEY (id_chaine, id_video)
    )
""")


cur.execute("""
    CREATE TABLE utilisateurs (
        id_utilisateur TEXT PRIMARY KEY,
        nom_utilisateur TEXT NOT NULL
    )
""")

cur.execute("""
    CREATE TABLE commentaires (
        id_commentaire TEXT PRIMARY KEY,
        contenu TEXT NOT NULL,
        date_commentaire DATE NOT NULL,
        id_video TEXT REFERENCES videos(id_video) ON DELETE CASCADE,
        id_utilisateur TEXT REFERENCES utilisateurs(id_utilisateur) ON DELETE CASCADE,
        id_commentaire_parent TEXT REFERENCES commentaires(id_commentaire) 
    )
""")


conn.commit()
cur.close()
conn.close()


# Préparer le JSON des chaînes

In [ ]:
channelsF3 = openJson("../../filtering/channels/jsons/channelsF3.json")
channelsF3Non = openJson("../../filtering/channels/jsons/channelsF3Non.json")
videosF3 = openJson("../../filtering/videos/jsons/videosF3.json")

channelsTable = channelsF3
len(channelsTable)

- Vérifier que l’ID d’une chaîne est unique

In [ ]:
verification = []
for channel in channelsTable:
    verification.append(channel['id_chaine'])

len(set(verification))

In [ ]:
count = 0
missedChannels = []

for video in videosF3:
    exist = False
    for channel in channelsTable:
        if video['id_chaine']==channel['id_chaine'] :
            exist = True
            break
    if not exist:
        missedChannels.append(video['id_chaine'])
        count +=1
count

In [ ]:
len(missedChannels)

In [ ]:
len(set(missedChannels))

- L’ID de chaîne peut être redondant, donc il faut travailler avec un `set()`.

In [ ]:
def getChannel(ID):
    for item in channelsF3Non:
        if ID == item['id_chaine']:
            return item

In [ ]:
for channelID in set(missedChannels):
    channelsTable.append(getChannel(channelID))
    
len(channelsTable)

In [ ]:
saveJson("../jsons/channelsTable.json",channelsTable)

# Enregistrer les chaînes


In [ ]:
channelsTable = openJson("../jsons/channelsTable.json")
len(channelsTable)

In [ ]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)
cur = conn.cursor()

for item in channelsTable:
    id_chaine = item["id_chaine"]
    nom = item["nom_chaine"]
    bio = item.get("bio", "")
    localisation = item.get("localisation", None)
    date_creation = item["date_creation"][:10]  # YYYY-MM-DD
    pertinente = item["decision"].strip().lower() == "oui"

    cur.execute("""
        INSERT INTO chaines (id_chaine, nom, bio, localisation, date_creation, pertinente)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (id_chaine) DO NOTHING
    """, (id_chaine, nom, bio, localisation, date_creation, pertinente))

conn.commit()
cur.close()
conn.close()

# ON CONFLICT DO NOTHING empêche les doublons (id_chaine déjà inséré)

print("Insertion terminée.")


# Enregistrer les vidéos

In [ ]:
videosF3 = openJson("../../filtering/videos/jsons/videosF3.json")
len(videosF3)

In [ ]:
import re

def toSeconds(iso_duration):
    # Exemple : "PT50M11S" → 3011 secondes
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', iso_duration)
    if not match:
        return None
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0
    return hours * 3600 + minutes * 60 + seconds

In [ ]:
toSeconds("PT1H4M1S")

In [ ]:
1*3600+4*60+1

In [ ]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)

cur = conn.cursor()


for video in videosF3:
    id_video = video["id_video"]
    id_chaine = video["id_chaine"]
    titre = video["titre_video"]
    description = video["description"]
    date_publication = video["date_publication"][:10]  # 'YYYY-MM-DD'
    duree = toSeconds(video["duree"])
    miniature = video["miniature"]
    langue = video["langue"][:2] if video["langue"] else None
    transcription = None
    tags = video["tags"] if isinstance(video["tags"], list) else None
    requetes = video["requete"] if isinstance(video["requete"], list) else None
    categorie_video = None  

    cur.execute("""
        INSERT INTO videos (
            id_video, titre, description, date_publication, categorie_video,
            duree, miniature, langue, transcription, tags, requetes, id_chaine
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (id_video) DO NOTHING
    """, (
        id_video, titre, description, date_publication, categorie_video,
        duree, miniature, langue, transcription, tags, requetes, id_chaine
    ))

conn.commit()
cur.close()
conn.close()


# Add missed data

In [2]:
from googleapiclient.discovery import build

youtube = build('youtube', 'v3', developerKey=os.getenv('YOUTUBE_API_Mono'))

In [3]:
def getProfileImage(channelID):
    request = youtube.channels().list(part="snippet",id=channelID)
    response = request.execute()
    try:
        thumbnails = response['items'][0]['snippet']['thumbnails']
        high_logo = thumbnails.get('high', {}).get('url')
        return high_logo
    except Exception as e:
        print(f'probleme with channel : {channelID} error {e}')
        return None

def updateChainesLogo():
    
    conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost", 
    port="5432" )

    cur = conn.cursor()
    cur.execute("select id_chaine from chaines")
    rows = cur.fetchall()
    #print(rows)
    
    for row in tqdm(rows):
        channelId = row[0]
        logo = getProfileImage(channelId)
        cur.execute("update chaines set logo = %s where id_chaine = %s ;",(logo,channelId))
        
    conn.commit()    
    cur.close()
    conn.close()


In [4]:
getProfileImage('UCVQeGg4Fdrrr8vDXa7yjOYg')

'https://yt3.ggpht.com/ytc/AIdro_lJv7-_a5amHuY5-kPXrKSBh4zInH2HJQhzD56O-Pm0-Vk=s800-c-k-c0x00ffffff-no-rj'

In [5]:
updateChainesLogo()

 22%|██▏       | 227/1017 [00:11<00:42, 18.47it/s]

probleme with channel : UCAZ9xwjoXb6yGVZZ1Whn4Fw error 'items'


 73%|███████▎  | 747/1017 [00:41<00:14, 18.14it/s]

probleme with channel : UCROJNgrAZh-BIapt1omtjEQ error 'items'


100%|██████████| 1017/1017 [00:56<00:00, 17.97it/s]
